In [ ]:
## RAG Hands-on tutorial
"""
Step 1: Vector Store
Step 2: Search the vector store and retrieve relevant documents
Step 3: Call LLM with the user query and the retrieved documents
Step 4: Return the LLM response to the user
"""
